In [1]:
from HMM_Functions import *
from matplotlib import cm, pyplot as plt
import seaborn
from matplotlib.dates import YearLocator, MonthLocator, DayLocator
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
tpm = [[0.3, 0.3, 0.2, 0.1, 0.05, 0.05],
       [0.25, 0.3, 0.25, 0.1, 0.05, 0.05],
       [0.1, 0.1, 0.3, 0.2, 0.2, 0.1],
       [0.1, 0.1, 0.2, 0.3, 0.2, 0.1],
       [0.05, 0.05, 0.1, 0.25, 0.3, 0.25],
       [0.05, 0.05, 0.1, 0.2, 0.3, 0.3]]

#tpm = create_TPM(6)

pi = [.2, .1, .1, .2, .1, .1]

epm = create_EPM(6, 2)
emissions = ['Decreasing', 'Decreasing','Increasing','Decreasing',
 'Decreasing','Decreasing','Decreasing', 'Decreasing', 'Increasing','Decreasing','Decreasing',
 'Decreasing','Increasing','Increasing','Decreasing','Decreasing']

In [30]:
'''
                            TPM          FWD PROB
P(Xt+k+1 | e1:t) = ∑t+k P(Xt+k+1|xt+k) P(xt+k|e1:t)


∑t+k P(Xt+k+1|xt+k) * P(xt+k|e1:t): Iterate through ALL the states and add up the TPM * most recent fwd probability

Outcome: matrix with probabilities for each of hte next states, prediction is the highest one:
          I    D
state 0: .05  .05  = .1  -> .5 .5 = 1
state 1: .05  .05  = .1  -> .5 .5 = 1
state 2: .1   .1  = .2   
state 3: .05  .25 = .3   -> .05 .95 = 1
state 4: .1.  .1  = .2
state 5: .05  .05 =.01

'''

def filtering(tpm, epm, emissions, pi, future_days=7):

    t = len(emissions)
    NUM_STATES = len(tpm)
    fwrd = forward(tpm, epm, pi, emissions)
    state_pred = []
    pred_list = []
    normalized_list = fwrd[t-1]
    
    #starting_fwd = fwd[T][state_i]
    #starting_fwd
    
    #∑t+k P(Xt+k+1|xt+k) * P(xt+k|e1:t): Iterate through ALL the states and add up the TPM * most recent fwd probability
    
    '''
    
   do we need to do this part?  
   EPM of I at state 0  * probability state 0| KNOW that we were just in 0 *  fwd of being in state 0 at previous time step
   EPM of D at state 0  * probability state 0| KNOW that we were just in 0 *  fwd of being in state 0 at previous time step
   
          probability state 0| KNOW that we were just in 1 *  fwd of state 1
          probability state 0| KNOW that we were just in 2 *  fwd of state 2
          probability state 0| KNOW that we were just in 3 *  fwd of state 3
          probability state 0| KNOW that we were just in 4 *  fwd of state 4
          probability state 0| KNOW that we were just in 5 *  fwd of state 5
            add all this up to get the probabiltiy of next state being state 0
            
          probability state 1| KNOW that we were just in 0 *  fwd of state 0
          probability state 1| KNOW that we were just in 1 *  fwd of state 1
          probability state 1| KNOW that we were just in 2 *  fwd of state 2
          probability state 1| KNOW that we were just in 3 *  fwd of state 3
          probability state 1| KNOW that we were just in 4 *  fwd of state 4
          probability state 1| KNOW that we were just in 5 *  fwd of state 5
            add all this up to get the probabiltiy of next state being state 1
            
          probability state 2| KNOW that we were just in 0 *  fwd of state 0
          probability state 2| KNOW that we were just in 1 *  fwd of state 1
          probability state 2| KNOW that we were just in 2 *  fwd of state 2
          probability state 2| KNOW that we were just in 3 *  fwd of state 3
          probability state 2| KNOW that we were just in 4 *  fwd of state 4
          probability state 2| KNOW that we were just in 5 *  fwd of state 5
            add all this up to get the probabiltiy of next state being state 2
        '''
    
    for iter in range(future_days):
        pred_list = []
        
        for state_i in range(NUM_STATES):
            pred_state_dist = tpm[state_i]
            total = 0

            for state_j in range(NUM_STATES):
                transition_prob = pred_state_dist[state_j]
                fwd_prob =  normalized_list[state_j]

                total = total + (transition_prob * fwd_prob)

            pred_list.append(total)

        denominator = 0
        normalized_list = []
        for i in range(len(pred_list)):
            denominator += pred_list[i]

        #print('denom is...', denominator)

        temp_list = []
        for i in range(len(pred_list)):
            value = pred_list[i]/denominator
            temp_list.append(value)

        normalized_list = normalized_list.clear()
        
        normalized_list = temp_list.copy()

        print('\nnormalized_list is..', normalized_list)

        checksum = sum(normalized_list)
        print('equals 1?',checksum)

        maxstate = np.argmax(normalized_list)
        state_pred.append(maxstate)
        print("max", maxstate)
    
    return(state_pred)
            
    

In [31]:
pred = filtering(tpm, epm, emissions, pi)
pred


normalized_list is.. [0.15173029775290853, 0.15479591185558508, 0.17400477849233528, 0.17511455356913824, 0.1736105761273977, 0.17074388220263512]
equals 1? 1.0
max 3

normalized_list is.. [0.16054427917234917, 0.16165149473137094, 0.1686819160098353, 0.16879224497744852, 0.17027366079586928, 0.17005640431312677]
equals 1? 0.9999999999999999
max 4

normalized_list is.. [0.1639803891842217, 0.16438650215748862, 0.16732618544784097, 0.16733719749617135, 0.16845331859455068, 0.16851640711972674]
equals 1? 1.0000000000000002
max 5

normalized_list is.. [0.16543646568816636, 0.16560363318996155, 0.1669221568242962, 0.1669232572240002, 0.1675277848502256, 0.1675867022233502]
equals 1? 1.0000000000000002
max 5

normalized_list is.. [0.1660920671876036, 0.1661663282963101, 0.16677687542587893, 0.16677698543111494, 0.16707729094001295, 0.16711045271907954]
equals 1? 1.0
max 5

normalized_list is.. [0.1663960521355977, 0.16643028762453696, 0.16671682927454948, 0.16671684027349148, 0.16686165986

[3, 4, 5, 5, 5, 5, 5]